In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]


In [9]:


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        print(f"embeds shape before reshaping: {self.embeddings(inputs).shape}")
        embeds = self.embeddings(inputs).view((1, -1))
        print(f"embeds shape after view function : {embeds.shape}")
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        print(f"context ids shape: {context_idxs.shape} ")

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        print(f"log probs shape after forward pass: {log_probs.shape}")
        print(f"log_probs: {log_probs}")
        print(f"target word is : {target}")
        print(f"target index is: {[word_to_ix[target]]} ")

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix["beauty"]])

context ids shape: torch.Size([2]) 
embeds shape before reshaping: torch.Size([2, 10])
embeds shape after view function : torch.Size([1, 20])
log probs shape after forward pass: torch.Size([1, 97])
log_probs: tensor([[-4.4198, -4.0783, -4.7874, -4.8897, -4.0923, -5.0492, -4.5195, -4.4847,
         -4.8235, -4.4609, -4.7630, -4.5476, -4.6811, -4.7654, -5.0643, -4.7324,
         -4.5814, -4.1848, -4.8280, -4.4023, -4.7455, -4.3362, -4.7263, -4.8628,
         -4.3984, -4.7615, -4.4350, -4.2321, -4.2178, -4.4724, -4.3562, -4.5951,
         -4.7008, -4.7020, -4.5191, -4.5236, -4.6918, -4.7598, -4.7326, -4.7027,
         -4.6900, -4.5302, -4.5079, -4.3595, -4.4304, -4.2689, -4.4659, -4.7245,
         -4.7363, -5.0334, -4.5351, -4.5905, -4.6966, -4.5326, -4.3569, -4.7695,
         -4.4256, -4.5159, -4.3019, -4.6803, -4.8774, -4.4400, -4.6056, -4.4515,
         -4.5509, -4.9959, -4.6232, -4.2461, -4.5705, -4.4081, -4.6676, -4.6423,
         -4.9145, -5.0592, -4.9387, -4.8583, -4.5148, -4.5656,